# Properties

Since Pydantic classes are regular Python classes, we can add methods and properties to our class.

For example:

In [1]:
from pydantic import BaseModel, Field, ValidationError

In [2]:
from math import pi


class Circle(BaseModel):
    center: tuple[int, int] = (0, 0)
    radius: int = Field(default=1, gt=0)

    def area(self):
        return pi * self.radius ** 2

We can create a model instance the usual way:

In [3]:
c = Circle(center=(1, 1), radius=2)

When we serialize our model, we just get our fields:

In [4]:
c.model_dump()

{'center': (1, 1), 'radius': 2}

But the instance has a method `area()`:

In [5]:
c.area()

12.566370614359172

We can also add properties to our class - let's use a property instead of a method for our area:

In [6]:
class Circle(BaseModel):
    center: tuple[int, int] = (0, 0)
    radius: int = Field(default=1, gt=0)

    @property
    def area(self):
        return pi * self.radius ** 2

Things work like before:

In [7]:
m = Circle()

In [8]:
m.area

3.141592653589793

But the property is neither represented not serialized:

In [9]:
m

Circle(center=(0, 0), radius=1)

In [10]:
m.model_dump()

{'center': (0, 0), 'radius': 1}

In [11]:
m.model_dump_json()

'{"center":[0,0],"radius":1}'

And, if we inspect the model fields, we won't see `area` in there either:

In [12]:
m.model_fields

{'center': FieldInfo(annotation=tuple[int, int], required=False, default=(0, 0)),
 'radius': FieldInfo(annotation=int, required=False, default=1, metadata=[Gt(gt=0)])}

So, while we can add properties to our pydantic models, they do not get serialized. We'll need to do a bit more work if we want those properties to be included in our model serialization.

Before we look at that, I want to point out that we could also choose to cache our properties.

In this example, we might want to cache the `area` - but that means we need to make sure the `radius` field is immutable (frozen). We could make the entire circle frozen, or we could choose to just make `radius` frozen.

I'll use the latter.

In [13]:
from functools import cached_property

class Circle(BaseModel):
    center: tuple[int, int] = (0, 0)
    radius: int = Field(default=1, gt=0, frozen=True)

    @cached_property
    def area(self):
        print("calculating area...")
        return pi * self.radius ** 2

In [14]:
c = Circle()

In [15]:
c.area

calculating area...


3.141592653589793

As you see, our property code was executed, but if we access the property again:

In [16]:
c.area

3.141592653589793

our property implementation is not executed again.

And this is OK, since `radius` is now frozen:

In [17]:
try:
    c.radius=2
except ValidationError as ex:
    print(ex)

1 validation error for Circle
radius
  Field is frozen [type=frozen_field, input_value=2, input_type=int]
    For further information visit https://errors.pydantic.dev/2.5/v/frozen_field


Ok, so as you can see we can add any method to our Pydantic classes, as well as properties and cached properties - but properties do not show up as model fields.